In [ ]:
# main libraries
import os
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# graphical libraries
from matplotlib import pyplot as plt
%matplotlib inline
from PIL import Image as mg
import cv2

# data science & deep learning libraries
import numpy as np
import tensorflow as tf



In [ ]:
# constants

WILD_PATH = 'wild'
IMAGE_SIZE = 128

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Getting Images

In [ ]:

images = os.listdir('/content/drive/MyDrive/imgs')
train_dir='/content/drive/MyDrive/imgs'

In [ ]:
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
train_ds = tf.keras.utils.image_dataset_from_directory(train_dir,validation_split=0.3,subset="training",seed=123,image_size=(IMAGE_SIZE, IMAGE_SIZE),)
val_ds = tf.keras.utils.image_dataset_from_directory(train_dir,validation_split=0.3,subset="validation",seed=123,image_size=(IMAGE_SIZE, IMAGE_SIZE),)
train_ds = train_ds.unbatch()
val_ds = val_ds.unbatch()
train_xs = []
test_xs = []
for x,y in train_ds:
    train_xs.append(x.astype('float32')/255.)
for x,y in val_ds:
    test_xs.append(x.astype('float32')/255.)
train_xs = tf.convert_to_tensor(train_xs)
test_xs = tf.convert_to_tensor(test_xs)

In [ ]:
print(train_xs.shape)
print(test_xs.shape)

### Show Random Train Image:

### Convolutional Layers:

In [ ]:
#generating masks
left = 60
right = 76
up = 78
down = 62
x_train = train_xs.numpy()
black_train = x_train.copy()
black_train[:,left:right,down:up,:] = 0
train_labels = x_train[:,left:right,down:up,:]
x_test = test_xs.numpy()
black_test = x_test.copy()
black_test[:,left:right,down:up,:] = 0
test_labels = x_test[:,left:right,down:up,:]
plt.imshow(black_test[0])

In [ ]:
from tensorflow.keras.models import Model

encoder_in = tf.keras.Input(shape=(128,128,3),name='img_input')
layer = tf.keras.layers.Conv2D(64,(3,3),activation='relu',padding='same')(encoder_in)
layer = tf.keras.layers.MaxPooling2D((2,2),padding='same')(layer)
layer = tf.keras.layers.Conv2D(64,(3,3),activation='relu',padding='same')(encoder_in)
layer = tf.keras.layers.MaxPooling2D((2,2),padding='same')(layer)
layer = tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding='same')(layer)
layer = tf.keras.layers.MaxPooling2D((2,2),padding='same')(layer)
layer = tf.keras.layers.Flatten()(layer)
encoder_out = tf.keras.layers.Dense(1024,activation='relu')(layer)     
decoder_in = tf.keras.layers.Dense(1024,activation='relu')(encoder_out)
layer = tf.keras.layers.Reshape((8,8,16))(decoder_in)
layer = tf.keras.layers.Conv2DTranspose(3,(3,3),activation='relu',padding='same')(layer)
layer = layer = tf.keras.layers.Conv2DTranspose(64,(3,3),activation='relu',padding='same')(layer)
layer = layer = tf.keras.layers.Conv2DTranspose(128,(3,3),activation='relu',padding='same')(layer)
layer = tf.keras.layers.Conv2DTranspose(3,(3,3),activation='relu',padding='same',strides=(2,2))(layer)
decoder_out = tf.keras.layers.Conv2DTranspose(3,(3,3),activation='relu',padding='same')(layer)

         
autoencoder = tf.keras.Model(encoder_in,decoder_out,name='an_dec')
autoencoder.summary()
      

In [ ]:

optimizer = tf.keras.optimizers.Adam(lr = 0.001,decay=1e-6)
autoencoder.compile(optimizer=optimizer, loss=tf.keras.losses.MeanSquaredError())

In [ ]:
history = autoencoder.fit(black_train,train_labels ,
                epochs=50,
                shuffle=True,
                batch_size=10,
                validation_data=(black_test,test_labels))

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
index = 100
ry = autoencoder.predict(black_test[index].reshape(-1,128,128,3))
pic = black_test[index].copy()
pic[left:right,down:up] = ry
rows = 1
colls = 3
fig = plt.figure(figsize=(10, 7))
fig.add_subplot(rows,colls,1)
plt.imshow(pic)
plt.axis('off')
plt.title("predicted")
fig.add_subplot(rows,colls,2)
plt.imshow(test_xs[index])
plt.axis('off')
plt.title("original")
fig.add_subplot(rows,colls,3)
plt.imshow(black_test[index])
plt.axis('off')
plt.title("input")

In [ ]:
#generating 32x32 holes
left = 52
right = 84
up = 86
down = 54
black_train_bigger = x_train.copy()
black_train_bigger[:,left:right,down:up,:] = 0
train_labels_bigger = x_train[:,left:right,down:up,:]
black_test_bigger = x_test.copy()
black_test_bigger[:,left:right,down:up,:] = 0
test_labels_bigger = x_test[:,left:right,down:up,:]
plt.imshow(black_train_bigger[0])

In [ ]:
from tensorflow.keras.models import Model

encoder_in = tf.keras.Input(shape=(128,128,3),name='img_input')
layer = tf.keras.layers.Conv2D(64,(3,3),activation='relu',padding='same')(encoder_in)
layer = tf.keras.layers.MaxPooling2D((2,2),padding='same')(layer)
layer = tf.keras.layers.Conv2D(64,(3,3),activation='relu',padding='same')(encoder_in)
layer = tf.keras.layers.MaxPooling2D((2,2),padding='same')(layer)
layer = tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding='same')(layer)
layer = tf.keras.layers.MaxPooling2D((2,2),padding='same')(layer)
layer = tf.keras.layers.Flatten()(layer)
encoder_out = tf.keras.layers.Dense(1024,activation='relu')(layer)
       
decoder_in = tf.keras.layers.Dense(1024,activation='relu')(encoder_out)
layer = tf.keras.layers.Reshape((8,8,16))(decoder_in)
layer = tf.keras.layers.Conv2DTranspose(3,(3,3),activation='relu',padding='same')(layer)
layer = layer = tf.keras.layers.Conv2DTranspose(64,(3,3),activation='relu',padding='same')(layer)
layer = tf.keras.layers.Conv2DTranspose(128,(3,3),activation='relu',padding='same')(layer)
layer = tf.keras.layers.Conv2DTranspose(3,(3,3),activation='relu',padding='same',strides=(2,2))(layer)
decoder_out = tf.keras.layers.Conv2DTranspose(3,(3,3),activation='relu',padding='same',strides=(2,2))(layer)

          
autoencoder = tf.keras.Model(encoder_in,decoder_out,name='an_dec')
autoencoder.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(lr = 0.001,decay=1e-6)
autoencoder.compile(optimizer=optimizer, loss=tf.keras.losses.MeanSquaredError())
history = autoencoder.fit(black_train_bigger,train_labels_bigger ,
                epochs=50,
                shuffle=True,
                batch_size=10,
                validation_data=(black_test_bigger,test_labels_bigger))


In [ ]:
index = 3
ry = autoencoder.predict(black_test_bigger[index].reshape(-1,128,128,3))
pic = black_test[index].copy()
pic[left:right,down:up] = ry
rows = 1
colls = 3
fig = plt.figure(figsize=(10, 7))
fig.add_subplot(rows,colls,1)
plt.imshow(pic)
plt.axis('off')
plt.title("predicted")
fig.add_subplot(rows,colls,2)
plt.imshow(test_xs[index])
plt.axis('off')
plt.title("original")
fig.add_subplot(rows,colls,3)
plt.imshow(black_test_bigger[index])
plt.axis('off')
plt.title("input")

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()